In [1]:
import yfinance as yf
import numpy as np
import pandas as pd

In [2]:
tsla = yf.download('TSLA', start='2019-01-01', end='2025-03-05')
xly = yf.download('XLY', start='2019-01-01', end='2025-03-05')
spy = yf.download('SPY', start='2019-01-01', end='2025-03-05')

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1551 entries, 2019-01-02 to 2025-03-04
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   (Close, SPY)   1551 non-null   float64
 1   (High, SPY)    1551 non-null   float64
 2   (Low, SPY)     1551 non-null   float64
 3   (Open, SPY)    1551 non-null   float64
 4   (Volume, SPY)  1551 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 72.7 KB


In [4]:
len(tsla)

1551

In [5]:
xly.head()

Price,Close,High,Low,Open,Volume
Ticker,XLY,XLY,XLY,XLY,XLY
Date,,,,,
2019-01-02,94.046127,94.574050,91.246237,91.745879,6840800
2019-01-03,92.009796,93.555862,91.849535,93.329610,6346000
2019-01-04,95.054817,95.752431,93.065670,93.254212,7269100
2019-01-07,97.204208,97.826402,95.460172,95.507303,6263100
2019-01-08,98.278923,98.806846,96.883689,98.212933,9391000


In [6]:
def calculate_vortex(df, value, n=14):
    high = df[("High", value)]
    low = df[("Low", value)]
    close = df[("Close", value)]

    # Calculate VM+ and VM-
    vm_plus = abs(high - low.shift(1))   # |Today's High - Yesterday's Low|
    vm_minus = abs(low - high.shift(1))  # |Today's Low - Yesterday's High|

    # Calculate True Range (TR)
    tr = pd.concat([
        high - low,
        abs(high - close.shift(1)),
        abs(low - close.shift(1))
    ], axis=1).max(axis=1)

    # Rolling sum for lookback period
    sum_vm_plus = vm_plus.rolling(window=n).sum()
    sum_vm_minus = vm_minus.rolling(window=n).sum()
    sum_tr = tr.rolling(window=n).sum()

    # Compute VI+ and VI-
    vi_plus = sum_vm_plus / sum_tr
    vi_minus = sum_vm_minus / sum_tr

    return vi_plus, vi_minus

In [7]:
tsla['VI+'], tsla['VI-'] = calculate_vortex(tsla, 'TSLA')
xly['VI+'], xly['VI-'] = calculate_vortex(xly, 'XLY')
spy['VI+'], spy['VI-'] = calculate_vortex(spy, 'SPY')

In [8]:
spy.head(20)

Price,Close,High,Low,Open,Volume,VI+,VI-
Ticker,SPY,SPY,SPY,SPY,SPY,,
Date,,,,,,,
2019-01-02,226.954727,227.889121,223.117420,223.144634,126925200,NaN,NaN
2019-01-03,221.538986,225.494230,221.049109,225.185783,144140700,NaN,NaN
2019-01-04,228.959595,229.612755,224.224188,224.605196,142628800,NaN,NaN
2019-01-07,230.764877,232.189121,228.324599,229.231765,103139100,NaN,NaN
2019-01-08,232.932999,233.422876,230.420159,232.978374,102512600,NaN,NaN
2019-01-09,234.021606,234.874345,232.406852,233.649665,95006600,NaN,NaN
2019-01-10,234.847137,235.101143,231.780912,232.470367,96823900,NaN,NaN
2019-01-11,234.937851,234.965065,233.168866,233.758519,73858100,NaN,NaN


In [9]:
import requests
import pandas as pd

url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey=PNM5EHRALIOT1CKJ'

response = requests.get(url)

if response.status_code == 200:
    sentiment_data = response.json()
    sentiment_df = pd.DataFrame(sentiment_data['feed']) 
    print(sentiment_df.head())
else:
    print("API call failed:", response.status_code)


                                               title  \
0  New Inflation Data Dismays Bulls, CoreWeave IP...   
1  Assessing Apple's Performance Against Competit...   
2  Apple's Options Frenzy: What You Need to Know ...   
3  Trading The SPY As PCE Inflation Report Takes ...   
4  EU To Issue Minimal Fines To Apple And Meta To...   

                                                 url   time_published  \
0  https://www.benzinga.com/markets/equities/25/0...  20250328T161558   
1  https://www.benzinga.com/insights/news/25/03/4...  20250328T150055   
2  https://www.benzinga.com/insights/options/25/0...  20250328T134533   
3  https://www.benzinga.com/markets/equities/25/0...  20250328T125805   
4  https://www.benzinga.com/markets/25/03/4453457...  20250328T122729   

               authors                                            summary  \
0   [The Arora Report]  To gain an edge, this is what you need to know...   
1  [Benzinga Insights]  Amidst the fast-paced and highly competitive b

In [10]:
sentiment_df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,"New Inflation Data Dismays Bulls, CoreWeave IP...",https://www.benzinga.com/markets/equities/25/0...,20250328T161558,[The Arora Report],"To gain an edge, this is what you need to know...",https://thearorareport.com/wp-content/uploads/...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.240809,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1725..."
1,Assessing Apple's Performance Against Competit...,https://www.benzinga.com/insights/news/25/03/4...,20250328T150055,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.8...",0.265078,Somewhat-Bullish,"[{'ticker': 'AAPL', 'relevance_score': '0.4680..."
2,Apple's Options Frenzy: What You Need to Know ...,https://www.benzinga.com/insights/options/25/0...,20250328T134533,[Benzinga Insights],Investors with a lot of money to spend have ta...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.185008,Somewhat-Bullish,"[{'ticker': 'EVR', 'relevance_score': '0.06568..."
3,Trading The SPY As PCE Inflation Report Takes ...,https://www.benzinga.com/markets/equities/25/0...,20250328T125805,[RIPS],Good Morning Traders! In today's Market Clubho...,https://www.benzinga.com/next-assets/images/sc...,Benzinga,Trading,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.084859,Neutral,"[{'ticker': 'MSFT', 'relevance_score': '0.0813..."
4,EU To Issue Minimal Fines To Apple And Meta To...,https://www.benzinga.com/markets/25/03/4453457...,20250328T122729,[Namrata Sen],The European Union ( EU ) is reportedly set to...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.103777,Neutral,"[{'ticker': 'META', 'relevance_score': '0.2658..."
5,3 Northern Mutual Funds for Solid Returns,https://www.zacks.com/stock/news/2436613/3-nor...,20250328T111100,[Zacks Equity Research],"Invest in Northern mutual funds like NOIEX, NM...",https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.272886,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1496..."
6,2 Breakout Growth Stocks You Can Buy and Hold ...,https://www.fool.com/investing/2025/03/28/2-br...,20250328T110700,[Anders Bylund],Looking for undervalued stocks with explosive ...,https://g.foolcdn.com/editorial/images/812691/...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.298788,Somewhat-Bullish,"[{'ticker': 'NFLX', 'relevance_score': '0.1478..."
7,Intel's SuperFluid Cooling Tech 'Possibly Suit...,https://www.benzinga.com/25/03/44532855/intels...,20250328T104410,[Rishabh Mishra],Intel Corp. INTC SuperFluid cooling tech could...,https://editorial-assets.benzinga.com/wp-conte...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",0.226797,Somewhat-Bullish,"[{'ticker': 'NVDA', 'relevance_score': '0.4078..."
8,Should Strive 500 ETF ( STRV ) Be on Your In...,https://www.zacks.com/stock/news/2436589/shoul...,20250328T102007,[Zacks Equity Research],Style Box ETF report for ...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Technology', 'relevance_score': '0...",0.183048,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.1331..."
9,Should Invesco Dividend Achievers ETF ( PFM )...,https://www.zacks.com/stock/news/2436592/shoul...,20250328T102005,[Zacks Equity Research],Style Box ETF report for ...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Financi

## volatility

In [11]:

# Flatten MultiIndex columns 
tsla.columns = [
    '_'.join(col).strip() if isinstance(col, tuple) else col
    for col in tsla.columns
]

# Calculate True Range
tsla["prev_close"] = tsla["Close_TSLA"].shift(1)
tsla["tr1"] = tsla["High_TSLA"] - tsla["Low_TSLA"]
tsla["tr2"] = abs(tsla["High_TSLA"] - tsla["prev_close"])
tsla["tr3"] = abs(tsla["Low_TSLA"] - tsla["prev_close"])

tsla["true_range"] = tsla[["tr1", "tr2", "tr3"]].max(axis=1)

# 10-day ATR
tsla["ATR_10"] = tsla["true_range"].rolling(window=10).mean()

# ---- STEP 4: Calculate ATR as a percentage of closing price ----
tsla["atr_pct"] = tsla["ATR_10"] / tsla["Close_TSLA"]

# allocating the capital

def position_size(row):
    if row["atr_pct"] < 0.03:  # < 3% volatility → low risk
        return 0.01  # allocate 1% of capital
    else:  # ≥ 3% volatility → high risk
        return 0.005  # allocate 0.5% of capital

tsla["position_size"] = tsla.apply(position_size, axis=1)

# ---- STEP 6: Optional - Capital allocation per trade ----
#capital = 100000 # Example: $100K total portfolio
#tsla["allocation_dollars"] = tsla["position_size"] * capital

# ---- Preview ----
print(tsla[["Close_TSLA", "ATR_10", "atr_pct", "position_size"]].tail(10))


            Close_TSLA     ATR_10   atr_pct  position_size
Date                                                      
2025-02-19  360.559998  16.703000  0.046325          0.005
2025-02-20  354.399994  16.464999  0.046459          0.005
2025-02-21  337.799988  17.021997  0.050391          0.005
2025-02-24  330.529999  16.770996  0.050740          0.005
2025-02-25  302.799988  18.879996  0.062351          0.005
2025-02-26  290.799988  18.412994  0.063318          0.005
2025-02-27  281.950012  18.257996  0.064756          0.005
2025-02-28  292.980011  18.067996  0.061670          0.005
2025-03-03  284.649994  19.281998  0.067739          0.005
2025-03-04  272.040009  20.654996  0.075926          0.005


In [14]:
import plotly.express as px
fig = px.line(tsla, x=tsla.index, y="atr_pct", title="ATR% Over Time")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [15]:
import plotly.express as px

# Filter only 2025 data
tsla_2025 = tsla[tsla.index.year == 2025]

# Plot
fig = px.line(tsla_2025, x=tsla_2025.index, y="atr_pct", title="ATR% Over Time (2025 Only)")
fig.add_hline(y=0.03, line_dash="dot", line_color="green", annotation_text="Low Volatility Cutoff")
fig.show()


In [39]:
# Calculate ATR percentage
tsla[('ATR_pct', '')] = tsla[('ATR_10', '')] / tsla[('Close', 'TSLA')]

# Create Buy Signal
tsla[('Buy_Signal', '')] = tsla[('VI_Cross_Up', '')]

# Create Sell Signal
# Basic: VI- crosses above VI+
tsla[('Sell_Signal', '')] = tsla[('VI_Cross_Down', '')]

# Optional: Trailing Stop-Loss 3%
# Track highest close since buy, and sell if price drops >3%
tsla[('Position', '')] = 0  # 1 = in trade, 0 = out of trade
peak_price = 0

for i in range(1, len(tsla)):
    if tsla[('Buy_Signal', '')].iloc[i]:
        tsla.at[tsla.index[i], ('Position', '')] = 1
        peak_price = tsla[('Close', 'TSLA')].iloc[i]
    elif tsla[('Position', '')].iloc[i-1] == 1:
        current_price = tsla[('Close', 'TSLA')].iloc[i]
        peak_price = max(peak_price, current_price)
        drawdown = (peak_price - current_price) / peak_price
        if drawdown >= 0.03:  # 3% trailing stop-loss
            tsla.at[tsla.index[i], ('Sell_Signal', '')] = True
            tsla.at[tsla.index[i], ('Position', '')] = 0
        else:
            tsla.at[tsla.index[i], ('Position', '')] = 1


In [40]:
print("Buy signals:", tsla[('Buy_Signal', '')].sum())
print("Sell signals:", tsla[('Sell_Signal', '')].sum())


Buy signals: 80
Sell signals: 135
